In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import optuna

In [16]:
physical_devices = tf.config.list_physical_devices()
tf.config.set_visible_devices(physical_devices[1], 'GPU')

In [17]:
BATCH_SIZE = 200
EPOCHS = 100
# CHECKPOINT_PATH = 'models/checkpoints/famous48_optuna_temp_best.keras'

CLASSES = 48
IMAGE_SIZE = 24

In [18]:
from sklearn.model_selection import train_test_split
import re

def load_dataset():

  def read_file(filename):
  
    with open(filename, 'r') as file:
      lines = file.readlines()
      
      # Remove newLines
      for i, line in enumerate(lines):
        lines[i] = line.replace('\n', '')
      
      # We assume these are integers
      EXAMPLES_NR = int(lines[0])
      PIXELS_NR = int(lines[1])
      
      inputs = list()
      labels = np.zeros(EXAMPLES_NR, dtype=int)
      
      examples_raw = lines[2:EXAMPLES_NR+2]
      
      for i, example_raw in enumerate(examples_raw):
        # Split by spaces (treats multiple as one)
        tokens = re.split('\s+', example_raw)
        
        pixel_values = np.array(tokens[0:PIXELS_NR])
        attributes = tokens[PIXELS_NR:]
        
        pixel_values = np.array(pixel_values, dtype=float)
        pixel_values = pixel_values.reshape([IMAGE_SIZE, IMAGE_SIZE])
        
        inputs.append(pixel_values)
        labels[i] = int(attributes[2])
        
      inputs = np.array(inputs)
    return inputs, labels

  # classes 0-15
  X_0, y_0 = read_file('./data/x24x24.txt')
  # classes 16-31
  X_1, y_1 = read_file('./data/y24x24.txt')
  # # classes 32-48
  X_2, y_2 = read_file('./data/z24x24.txt')
  
  # Concatenate train and test images
  X = np.concatenate((X_0, X_1, X_2))
  y = np.concatenate((y_0, y_1, y_2))
  print(len(X), len(y))

  N_TRAIN_EXAMPLES=int(len(X) * 0.8)
  N_TEST_EXAMPLES=len(X) - N_TRAIN_EXAMPLES


  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

  # One-Hot encoding
  # Getting dummy variables
  y_train_fixed = np.zeros((y_train.shape[0], CLASSES))
  y_test_fixed = np.zeros((y_test.shape[0], CLASSES))

  for i, value in enumerate(y_train):
    y_train_fixed[i][value] = 1
    
  for i, value in enumerate(y_test):
    y_test_fixed[i][value] = 1
    
  return X_train, X_test, y_train_fixed, y_test_fixed

In [19]:
from keras import layers, regularizers, Sequential, Input
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy

def define_model(trial):

  conv_regularizer = regularizers.l2(l=trial.suggest_float("conv_regularizer", 1e-5, 1e-2)) # 0.0008
  dense_regularizer = regularizers.l2(l=trial.suggest_float("dense_regularizer", 1e-5, 1)) # 0.01

  activation_def = trial.suggest_categorical("activation_def", ["tanh", "relu"])
  # activation_output = trial.suggest_categorical("activation_output", ["tanh", "relu", ""])
  model = Sequential(
    [
      Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
      layers.Conv2D(6, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      layers.MaxPooling2D(pool_size=2, strides=2),
      
      layers.Dropout(trial.suggest_float("dropout_3", 0, 0.3)),
      layers.Conv2D(16, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      layers.MaxPooling2D(pool_size=2, strides=2),
      
      layers.Dropout(trial.suggest_float("dropout_2", 0, 0.5)),
      layers.Conv2D(120, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      
      layers.Flatten(),
      layers.Dropout(trial.suggest_float("dropout_1", 0, 0.65)),
      layers.Dense(84, activation=activation_def, kernel_regularizer=dense_regularizer),
      layers.Dense(CLASSES, activation='softmax'),
    ]
  )
  
  optimizer = Adam(learning_rate=0.001)

  model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=["accuracy"])
  
  return model


# Defines training and evaluation.
def train_model(model, X_train, y_train):
  
  history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    shuffle=True,
    verbose=0,
  )
  
  return history


def evaluate_trial(history):
  MEASUREMENT_SPAN = 15
  length = len(history.history['accuracy'])
  
  acc = np.mean(history.history['accuracy'][length-MEASUREMENT_SPAN:])
  val_acc = np.mean(history.history['val_accuracy'][length-MEASUREMENT_SPAN:])

  # Should be minimized
  difference = acc - val_acc
  
  return val_acc, difference

In [20]:
X_train, X_test, y_train, y_test = load_dataset()

def objective(trial):  
  model = define_model(trial)
  
  history = train_model(model, X_train, y_train)
  val_acc, difference = evaluate_trial(history)
  return val_acc, difference

6835 6835


In [21]:
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=30, timeout=3600)

print("Number of finished trials: ", len(study.trials))

[I 2024-04-30 16:06:30,083] A new study created in memory with name: no-name-47fb37a7-0637-4b21-8359-df08a6a18e09
c:\Users\Pawel\anaconda3\envs\tf_gpu_2\lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "conv_regularizer"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 1e-05, 'high': 0.01, 'log': False}
  warnings.warn(
[I 2024-04-30 16:07:19,164] Trial 0 finished with values: [0.7989640394846599, 0.07808214028676341] and parameters: {'conv_regularizer': 0.003527722871497799, 'activation_def': 'relu', 'dropout_3': 0.03363272229073839, 'dropout_2': 0.19798290147171926, 'dropout_1': 0.46969125121149846}. 
[I 2024-04-30 16:08:07,165] Trial 1 finished with values: [0.8175502777099609, 0

KeyboardInterrupt: 